In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/unsmile/unsmile_dataset.tsv',delimiter='\t',on_bad_lines=
'skip')

In [3]:
df.head(10)

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...,0,1,0,0,0,0,0,0,0,0,0
1,여자들은 취미가 애낳는건가.. 취미를 좀 가져라,1,0,0,0,0,0,0,0,0,0,0
2,개슬람녀 다 필요없고 니 엄마만 있으면 된다,0,0,0,1,0,0,1,0,0,0,0
3,조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...,0,1,0,0,0,0,0,0,0,0,0
4,바지 내리다 한남들 와꾸 보고 올려뿟노,0,1,0,0,0,0,0,0,0,0,0
5,저새끼 슈스케나온새끼잖아 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,0,0,0,0,0,0,0,0,1,0,0
6,모든 이슬람이 테러리스트는 아니지만 모든 테러리스트는 이슬람이다. 자유민주국가에서 ...,0,0,0,1,0,0,1,0,0,0,0
7,또 남자네...? 역시 한남충들은... (절레절레),0,1,0,0,0,0,0,0,0,0,0
8,그만해라 틀딱새끼들 달려와서 너 좆족으로 몰아간다 이제,0,0,0,1,1,0,0,0,0,0,0
9,잘생긴 형같음절대로 여자로 안보임 ㅇㅇ,1,0,0,0,0,0,0,0,0,0,0


In [ ]:

from soynlp.normalizer import *
# 정제된 텍스트를 저장할 리스트 초기화
cleaned_corpus = []
# 각 문장에 대해 이모티콘 정제 적용
for sentence in corpus:
    cleaned_sentence = emoticon_normalize(sentence, num_repeats=3)
    cleaned_corpus.append(cleaned_sentence)